In [1]:
import gym
import random
import numpy as np
from statistics import mean, median
from collections import Counter
import tensorflow as tf
tf.disable_eager_execution()
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
LR = 1e-3
env = gym.make('CartPole-v1')
env.reset()
TRIALS = 20
params = dict(
    goal_steps = 500,
    score_requirement = 30,
    initial_games = 500,
    X = np.array([[]]),
    Y = np.array([]),
    action = 2,
    score = np.array([0]),
    plot = False,
    model = None
)

CartPoleEnv - Version 0.2.0, Noise case: 1


/home/anany/anaconda3/envs/ML/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Environment '<class 'gym.envs.classic_control.cartpole.CartPoleEnv'>' has deprecated methods '_step' and '_reset' rather than 'step' and 'reset'. Compatibility code invoked. Set _gym_disable_underscore_compat = True to disable this behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [3]:
def plot(scores=[]):
    plt.clf()
    avg_scores = [sum(scores[:index+1])/(index+1) for index, 
                  score in enumerate(scores)]
    avg_score = sum(scores)/len(scores)
    x_val = [0,len(scores)-1]
    y_val = [avg_score, avg_score]
    plt.plot(scores,'g-', label='current score')
    plt.plot(x_val, y_val,'r-', label='average score')
    plt.plot(avg_scores,'b-', label='average scores')
    plt.xlabel('Episodes--->')
    plt.ylabel('Score--->')
    plt.legend()
    plt.plot()

In [4]:
def data(params):
    env.reset()
    for game in range(params["initial_games"]):
        env.reset()
        game_data = np.array([])
        prev_observation = np.array([])
        action_list = np.array([])
        for step in range(params["goal_steps"]):
            if not prev_observation.size or params['model'] is None:
                action = env.action_space.sample()
            else :
                prob_values = model.predict(np.array([prev_observation]))[0]
                action = np.argmax(prob_values)
            observation, reward, done, info =  env.step(action)
            params['score'][-1] += reward
            if prev_observation.size :
                if not game_data.size:
                    game_data = prev_observation
                    action_list = np.array([action])
                else:
                    game_data = np.vstack((game_data, prev_observation))
                    action_list = np.concatenate((action_list,action), axis = None)
            if done:
                break
            prev_observation = observation
        if params['plot']:
            plot(params['score'])
        if params['score'][-1] >= params['score_requirement']:
            if not params['X'].size:
                params['X'] = game_data
                params['Y'] = action_list
            else:
                params['X']=np.concatenate((params['X'], game_data), axis=0)
                params['Y']=np.concatenate((params['Y'], action_list))
        params['score'] = np.concatenate((params['score'], 0), axis = None)
data(params)

In [5]:
model = tf.keras.Sequential([tf.keras.layers.Dense(64,input_shape=(4,),activation='relu'),
                             tf.keras.layers.Dense(128,activation='relu'),
                             tf.keras.layers.Dense(2, activation='softmax'),

])
final_score_list = params['score']
for trial in range(TRIALS):    
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
    if params['X'].size > 0:
        model.fit(params['X'],params['Y'], epochs=10)
    else:
        break
    params = dict(
        goal_steps = 500,
        score_requirement = 50 + trial * 20,
        initial_games = 100,
        X = np.array([[]]),
        Y = np.array([]),
        action = 2,
        score = np.array([0]),
        plot = False,
        model = model
    )
    data(params)
    final_score_list = np.concatenate((final_score_list, params['score']), axis=None)
plot(final_score_list)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 931 samples
Epoch 1/10
931/931 [==============================] - 0s 214us/sample - loss: 0.6902
Epoch 2/10
931/931 [==============================] - 0s 67us/sample - loss: 0.6783
Epoch 3/10
931/931 [==============================] - 0s 61us/sample - loss: 0.6737
Epoch 4/10
931/931 [==============================] - 0s 52us/sample - loss: 0.6696
Epoch 5/10
931/931 [==============================] - 0s 51us/sample - loss: 0.6659
Epoch 6/10
931/931 [==============================] - 0s 51us/sample - loss: 0.6603
Epoch 7/10
931/931 [==============================] - 0s 40us/sample - loss: 0.6569
Epoch 8/10
931/931 [==============================] - 0s 44us/sample - loss: 0.6563
Epoch 9/10
931/931 [==============================] - 0s 52us/sample - loss: 0.6530
Epoch 10/10
931/931 [==============================] - 0s 46us/sample - loss: 0.6498
Train on 18963 samples
Epoch 1/10
18963/18963 [=========

6660/6660 [==============================] - 0s 59us/sample - loss: 0.0200
Epoch 8/10
6660/6660 [==============================] - 0s 42us/sample - loss: 0.0226
Epoch 9/10
6660/6660 [==============================] - 0s 58us/sample - loss: 0.0216
Epoch 10/10
6660/6660 [==============================] - 0s 52us/sample - loss: 0.0178
Train on 3834 samples
Epoch 1/10
3834/3834 [==============================] - 0s 101us/sample - loss: 0.0284
Epoch 2/10
3834/3834 [==============================] - 0s 39us/sample - loss: 0.0199
Epoch 3/10
3834/3834 [==============================] - 0s 39us/sample - loss: 0.0216
Epoch 4/10
3834/3834 [==============================] - 0s 40us/sample - loss: 0.0241
Epoch 5/10
3834/3834 [==============================] - 0s 57us/sample - loss: 0.0149
Epoch 6/10
3834/3834 [==============================] - 0s 80us/sample - loss: 0.0170
Epoch 7/10
3834/3834 [==============================] - 0s 47us/sample - loss: 0.0158
Epoch 8/10
3834/3834 [===================

387/387 [==============================] - 0s 51us/sample - loss: 0.0034
Train on 1362 samples
Epoch 1/10
1362/1362 [==============================] - 0s 342us/sample - loss: 0.0309
Epoch 2/10
1362/1362 [==============================] - 0s 55us/sample - loss: 0.0088
Epoch 3/10
1362/1362 [==============================] - 0s 55us/sample - loss: 0.0072
Epoch 4/10
1362/1362 [==============================] - 0s 51us/sample - loss: 0.0150
Epoch 5/10
1362/1362 [==============================] - 0s 51us/sample - loss: 0.0184
Epoch 6/10
1362/1362 [==============================] - 0s 58us/sample - loss: 0.0078
Epoch 7/10
1362/1362 [==============================] - 0s 53us/sample - loss: 0.0124
Epoch 8/10
1362/1362 [==============================] - 0s 51us/sample - loss: 0.0079
Epoch 9/10
1362/1362 [==============================] - 0s 84us/sample - loss: 0.0087
Epoch 10/10
1362/1362 [==============================] - 0s 84us/sample - loss: 0.0115
